## Setup

In [4]:
import os
from google.cloud import storage
from google.cloud import vision


def authenticate_implicit_with_adc(project_id="argon-producer-437209-a7"):
    """
    When interacting with Google Cloud Client libraries, the library can auto-detect the
    credentials to use.

    // TODO(Developer):
    //  1. Before running this sample,
    //  set up ADC as described in https://cloud.google.com/docs/authentication/external/set-up-adc
    //  2. Replace the project variable.
    //  3. Make sure that the user account or service account that you are using
    //  has the required permissions. For this sample, you must have "storage.buckets.list".
    Args:
        project_id: The project id of your Google Cloud project.

    Basically, run:
        1. gcloud init
        2. gcloud auth application-default login
    At each step, login via browser. This will save credentials to $HOME/.config/gcloud/application_default_credentials.json
    """

    # This snippet demonstrates how to list buckets.
    # *NOTE*: Replace the client created below with the client required for your application.
    # Note that the credentials are not specified when constructing the client.
    # Hence, the client library will look for credentials using ADC.
    storage_client = storage.Client(project=project_id)
    buckets = storage_client.list_buckets()
    print("Buckets:")
    for bucket in buckets:
        print(bucket.name)
    print("Listed all storage buckets.")

In [2]:
authenticate_implicit_with_adc()

Buckets:
quo-trial
Listed all storage buckets.


In [23]:
storage_client = storage.Client(project=project_id)

In [25]:
bucket = storage_client.get_bucket('quo-trial')

In [27]:
blobs = storage_client.list_blobs('quo-trial')

files = []
# Note: The call returns a response only when the iterator is consumed.
for blob in blobs:
    files.append(blob.name)


In [36]:
with open('reference_images2.csv', 'w+') as f :
    for x in files:
        if x.endswith('.jpg') or x.endswith('png'):
            parts = x.split('/')
            f.writelines(f"gs://quo-trial/{x},,master_set,{parts[1]},apparel-v2,{parts[1]},\"style=women,category=tops\",\n")

In [6]:
project_id = "argon-producer-437209-a7"
location = "us-east1"
product_set_id = "master_set"  # "lovebonito-top"
product_category = "apparel-v2"

## Index

In [21]:
def import_product_sets(project_id, location, gcs_uri):
    """Import images of different products in the product set.
    Args:
        project_id: Id of the project.
        location: A compute region name.
        gcs_uri: Google Cloud Storage URI.
            Target files must be in Product Search CSV format.
    """
    client = vision.ProductSearchClient()

    # A resource that represents Google Cloud Platform location.
    location_path = f"projects/{project_id}/locations/{location}"

    # Set the input configuration along with Google Cloud Storage URI
    gcs_source = vision.ImportProductSetsGcsSource(csv_file_uri=gcs_uri)
    input_config = vision.ImportProductSetsInputConfig(gcs_source=gcs_source)

    # Import the product sets from the input URI.
    response = client.import_product_sets(
        parent=location_path, input_config=input_config
    )

    print(f"Processing operation name: {response.operation.name}")
    # synchronous check of operation status
    result = response.result()
    print("Processing done.")

    for i, status in enumerate(result.statuses):
        print("Status of processing line {} of the csv: {}".format(i, status))
        # Check the status of reference image
        # `0` is the code for OK in google.rpc.Code.
        if status.code == 0:
            reference_image = result.reference_images[i]
            print(reference_image)
        else:
            print(f"Status code not OK: {status.message}")

In [37]:
project_id = "argon-producer-437209-a7"
location = "us-east1"
gcs_uri = "gs://quo-trial/reference_images.csv"
import_product_sets(project_id, location, gcs_uri)

Processing operation name: projects/argon-producer-437209-a7/locations/us-east1/operations/08942a8117087a76
Processing done.
Status of processing line 0 of the csv: 
name: "projects/argon-producer-437209-a7/locations/us-east1/products/aara-ruched-tie-back-top/referenceImages/618165782782179958-0"
uri: "gs://quo-trial/lovebonito_top/aara-ruched-tie-back-top/hy5963-031_umvulxp1vy9wjl6k.jpg"

Status of processing line 1 of the csv: 
name: "projects/argon-producer-437209-a7/locations/us-east1/products/abbye-printed-high-neck-top/referenceImages/618165782782179958-1"
uri: "gs://quo-trial/lovebonito_top/abbye-printed-high-neck-top/fm0207-005-1_4.jpg"

Status of processing line 2 of the csv: 
name: "projects/argon-producer-437209-a7/locations/us-east1/products/abbye-printed-high-neck-top/referenceImages/618165782782179958-2"
uri: "gs://quo-trial/lovebonito_top/abbye-printed-high-neck-top/fm0207-005_3.jpg"

Status of processing line 3 of the csv: 
name: "projects/argon-producer-437209-a7/locat

IndexError: list index (3337) out of range

In [39]:
client = vision.ProductSearchClient()
product_sets = client.list_product_sets(parent=f"projects/{project_id}/locations/{location}")
for product_set in product_sets:
    print(f"Product set name: {product_set.name}")
    print("Product set id: {}".format(product_set.name.split("/")[-1]))
    print(f"Product set display name: {product_set.display_name}")
    print("Product set index time: ")
    print(product_set.index_time)

Product set name: projects/argon-producer-437209-a7/locations/us-east1/productSets/master_set
Product set id: master_set
Product set display name:  
Product set index time: 
2024-10-09 11:54:34.558044+00:00


In [8]:
def delete_product_set(project_id, location, product_set_id):
    """Delete a product set.
    Args:
        project_id: Id of the project.
        location: A compute region name.
        product_set_id: Id of the product set.
    """
    client = vision.ProductSearchClient()

    # Get the full path of the product set.
    product_set_path = client.product_set_path(
        project=project_id, location=location, product_set=product_set_id
    )

    # Delete the product set.
    client.delete_product_set(name=product_set_path)
    print("Product set deleted.")

In [38]:
product_sets = client.list_product_sets(parent=f"projects/{project_id}/locations/{location}")
for product_set in product_sets:
    delete_product_set(project_id, location, product_set.name.split('/')[-1])

Product set deleted.


## Search

In [41]:


def get_similar_products_file(
    project_id,
    location,
    product_set_id,
    product_category,
    file_path,
    filter,
    max_results,
):
    """Search similar products to image.
    Args:
        project_id: Id of the project.
        location: A compute region name.
        product_set_id: Id of the product set.
        product_category: Category of the product.
        file_path: Local file path of the image to be searched.
        filter: Condition to be applied on the labels.
                Example for filter: (color = red OR color = blue) AND style = kids
                It will search on all products with the following labels:
                color:red AND style:kids
                color:blue AND style:kids
        max_results: The maximum number of results (matches) to return. If omitted, all results are returned.
    """
    # product_search_client is needed only for its helper methods.
    product_search_client = vision.ProductSearchClient()
    image_annotator_client = vision.ImageAnnotatorClient()

    # Read the image as a stream of bytes.
    with open(file_path, "rb") as image_file:
        content = image_file.read()

    # Create annotate image request along with product search feature.
    image = vision.Image(content=content)

    # product search specific parameters
    product_set_path = product_search_client.product_set_path(
        project=project_id, location=location, product_set=product_set_id
    )
    product_search_params = vision.ProductSearchParams(
        product_set=product_set_path,
        product_categories=[product_category],
        filter=filter,
    )
    image_context = vision.ImageContext(product_search_params=product_search_params)

    # Search products similar to the image.
    response = image_annotator_client.product_search(
        image, image_context=image_context, max_results=max_results
    )

    index_time = response.product_search_results.index_time
    print("Product set index time: ")
    print(index_time)

    results = response.product_search_results.results

    print("Search results:")
    for result in results:
        product = result.product

        print(f"Score(Confidence): {result.score}")
        print(f"Image name: {result.image}")

        print(f"Product name: {product.name}")
        print("Product display name: {}".format(product.display_name))
        print(f"Product description: {product.description}\n")
        print(f"Product labels: {product.product_labels}\n")

In [42]:
project_id = "argon-producer-437209-a7"
location = "us-east1"
product_set_id = "master_set"  # "lovebonito-top"
product_category = "apparel-v2"
file_path = (
    "/Users/kenjilaurens/Desktop/loveflair ig/Screenshot 2024-09-26 at 12.26.18 AM.png"
)
filter = ""
max_results = 10

sim = get_similar_products_file(
    project_id=project_id,
    location=location,
    product_set_id=product_set_id,
    product_category=product_category,
    file_path=file_path,
    filter=filter,
    max_results=max_results,
)

Product set index time: 
2024-10-09 11:24:38.899254+00:00
Search results:
Score(Confidence): 0.48991531133651733
Image name: projects/argon-producer-437209-a7/locations/us-east1/products/lunox-vest-in-butter/referenceImages/223404911869558334-2778
Product name: projects/argon-producer-437209-a7/locations/us-east1/products/lunox-vest-in-butter
Product display name: lunox-vest-in-butter
Product description: 

Product labels: [key: "category"
value: "tops"
]

Score(Confidence): 0.47395774722099304
Image name: projects/argon-producer-437209-a7/locations/us-east1/products/omara-vest-in-oat/referenceImages/8297563069404136186-2916
Product name: projects/argon-producer-437209-a7/locations/us-east1/products/omara-vest-in-oat
Product display name: omara-vest-in-oat
Product description: 

Product labels: [key: "category"
value: "tops"
]

Score(Confidence): 0.4647291302680969
Image name: projects/argon-producer-437209-a7/locations/us-east1/products/charles-vest-in-broken-white/referenceImages/829